In [1]:
from dotenv import load_dotenv
import os
from utils.s3_download import list_all_objects, s3_client
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from multiprocessing import Pool


load_dotenv(override=True)

BUCKET_NAME = os.getenv("BUCKET_NAME")
RAW_FOLDER = os.getenv("RAW_FOLDER")
CLEAN_FOLDER = os.getenv("CLEAN_FOLDER")
SELECTED_FOLDER = os.getenv("SELECTED_FOLDER")
LOCAL_BUCKET_FOLDER = os.getenv("LOCAL_BUCKET_FOLDER")

IMAGE_SIZE = eval(os.getenv("IMAGE_SIZE"))

source_folder = RAW_FOLDER

s3_files = list_all_objects(s3_client, BUCKET_NAME, source_folder); print(f"Number of files in {BUCKET_NAME}/{source_folder}: {len(s3_files)}")
s3_files = [k['Key'] for k in s3_files if not k['Key'].endswith('/')]
s3_files[:3]

listing s3 objects: 41it [00:06,  5.87it/s]

Number of files in sg-implement/prod_raw: 40941


['prod_raw/PS1053/flail_mower_2023-05-23-15-02-13_1.jpg',
 'prod_raw/PS1053/flail_mower_2023-05-23-15-02-13_1.json',
 'prod_raw/PS1053/flail_mower_2023-05-23-15-02-13_2.jpg']

In [7]:
def download_and_resize(file:str, s3_client=s3_client):
    local_save_folder = Path(f"{LOCAL_BUCKET_FOLDER}")
    local_img_path = local_save_folder / file
    if local_img_path.exists():
        return False
    local_img_path.parent.mkdir(parents=True, exist_ok=True)
    s3_client.download_file(BUCKET_NAME, file, str(local_img_path))
    if local_img_path.suffix == '.jpg':
        img = Image.open(local_img_path)
        img = img.resize(IMAGE_SIZE)
        img.save(local_img_path)
    return True

def download_images(s3_files:list[str]):
    with Pool(32) as p:
        r = list(tqdm(p.imap(download_and_resize, s3_files), total=len(s3_files)))
    num_downloaded = sum(r)
    print(f"Number of files downloaded: {num_downloaded}")
    return num_downloaded

num_downloaded = download_images(s3_files)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40938/40938 [00:08<00:00, 4910.22it/s]

Number of files downloaded: 1445
